
# Raspberry Pi Webcam Capture & Labeling (Logitech C270, 720p)

This notebook gives you an in-notebook interface to:
- **Preview** your Logitech C270 camera (1280×720)
- **Capture** images with a selected **class label**
- **Review & relabel** captured images

It stores images under `datasets/raw/<label>/IMG_...jpg` and writes a `datasets/raw/manifest.csv` with columns: `filepath,label,timestamp`.

> If you haven't already, install dependencies on your Pi:
```bash
sudo apt update
sudo apt install -y python3-opencv
pip install ipywidgets
jupyter nbextension enable --py widgetsnbextension
```


## 0) Imports & Paths

In [ ]:

import os, csv, time, threading, io, glob
from pathlib import Path
from datetime import datetime

import cv2
import numpy as np

from IPython.display import display, clear_output
import ipywidgets as widgets

# Root dataset folder
DATA_ROOT = Path('datasets/raw')
DATA_ROOT.mkdir(parents=True, exist_ok=True)

MANIFEST = DATA_ROOT / 'manifest.csv'
if not MANIFEST.exists():
    with open(MANIFEST, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['filepath','label','timestamp'])

print('Dataset root:', DATA_ROOT.resolve())
print('Manifest:', MANIFEST.resolve())



## 1) Live Camera Preview + Capture
- Choose a **label** from the dropdown (edit/add to the list as you wish).
- Press **Start Preview** to see the live feed.
- Press **Capture** to save a frame to the selected label folder.
- Press **Stop Preview** to release the camera.


In [ ]:

# --- User-configurable defaults ---
DEFAULT_LABELS = ['yellow', 'purple', 'background']  # edit as needed
CAMERA_INDEX = 0
FRAME_WIDTH = 1280
FRAME_HEIGHT = 720
JPEG_QUALITY = 90
# ----------------------------------

# Widgets
label_dd = widgets.Dropdown(options=DEFAULT_LABELS, value=DEFAULT_LABELS[0], description='Label:', layout=widgets.Layout(width='250px'))
new_label_txt = widgets.Text(placeholder='Add new label', description='New label:')
add_label_btn = widgets.Button(description='Add Label', button_style='')
start_btn = widgets.Button(description='Start Preview', button_style='success')
stop_btn = widgets.Button(description='Stop Preview', button_style='warning', disabled=True)
capture_btn = widgets.Button(description='Capture', button_style='primary', disabled=True)
status_out = widgets.Output(layout={'border': '1px solid #ccc'})
image_widget = widgets.Image(format='jpg', width=640, height=360)

controls = widgets.HBox([label_dd, new_label_txt, add_label_btn, start_btn, stop_btn, capture_btn])
ui = widgets.VBox([controls, image_widget, status_out])

display(ui)

# State
cap = None
preview_running = False

def log(msg):
    with status_out:
        print(msg)

def add_label_clicked(_):
    txt = new_label_txt.value.strip()
    if txt and txt not in label_dd.options:
        label_dd.options = list(label_dd.options) + [txt]
        label_dd.value = txt
        new_label_txt.value = ''
        log(f'[INFO] Added label: {txt}')
    elif txt:
        log(f'[WARN] Label "{txt}" already exists.')

add_label_btn.on_click(add_label_clicked)

def start_preview(_):
    global cap, preview_running
    if preview_running:
        log('[WARN] Preview already running.')
        return
    cap = cv2.VideoCapture(CAMERA_INDEX)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, FRAME_WIDTH)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, FRAME_HEIGHT)
    if not cap.isOpened():
        log('[ERROR] Could not open camera index %d' % CAMERA_INDEX)
        return
    preview_running = True
    start_btn.disabled = True
    stop_btn.disabled = False
    capture_btn.disabled = False

    def loop():
        global preview_running
        while preview_running:
            ok, frame = cap.read()
            if not ok:
                log('[WARN] Failed to read frame.')
                time.sleep(0.05)
                continue
            overlay = frame.copy()
            cv2.putText(overlay, f'label: {label_dd.value}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255,255,255), 2)
            cv2.putText(overlay, f'{FRAME_WIDTH}x{FRAME_HEIGHT}', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
            # Encode to JPEG for display
            encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), JPEG_QUALITY]
            ret, jpg = cv2.imencode('.jpg', overlay, encode_param)
            if ret:
                image_widget.value = jpg.tobytes()
            time.sleep(0.03)  # ~30 FPS ceiling

    threading.Thread(target=loop, daemon=True).start()
    log('[INFO] Preview started.')

def stop_preview(_):
    global cap, preview_running
    preview_running = False
    start_btn.disabled = False
    stop_btn.disabled = True
    capture_btn.disabled = True
    if cap is not None:
        cap.release()
        cap = None
    log('[INFO] Preview stopped.')

def capture_frame(_):
    global cap
    if cap is None:
        log('[ERROR] Camera not started.')
        return
    ok, frame = cap.read()
    if not ok:
        log('[ERROR] Could not capture frame.')
        return
    label = label_dd.value
    label_dir = DATA_ROOT / label
    label_dir.mkdir(parents=True, exist_ok=True)
    ts = datetime.now().strftime('%Y%m%d_%H%M%S_%f')[:-3]  # ms
    fname = f'IMG_{ts}.jpg'
    fpath = label_dir / fname
    cv2.imwrite(str(fpath), frame, [int(cv2.IMWRITE_JPEG_QUALITY), JPEG_QUALITY])
    with open(MANIFEST, 'a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([str(fpath), label, ts])
    log(f'[SAVED] {fpath}')

start_btn.on_click(start_preview)
stop_btn.on_click(stop_preview)
capture_btn.on_click(capture_frame)



## 2) Review & Relabel Images
- Use **Load List** to scan your dataset.
- Navigate with **Prev/Next**.
- Change the **label dropdown** and click **Save Label** to update the manifest **and** move the file to the label’s folder.
- Use **Delete Image** to remove a bad capture.


In [ ]:

# Widgets for review UI
load_btn = widgets.Button(description='Load List', button_style='success')
prev_btn = widgets.Button(description='Prev', button_style='')
next_btn = widgets.Button(description='Next', button_style='')
save_label_btn = widgets.Button(description='Save Label', button_style='primary')
delete_btn = widgets.Button(description='Delete Image', button_style='danger')
review_label_dd = widgets.Dropdown(options=list(label_dd.options), description='Label:', layout=widgets.Layout(width='250px'))
idx_label = widgets.Label('Index: - / -')
review_img = widgets.Image(format='jpg', width=640, height=360)
review_out = widgets.Output(layout={'border': '1px solid #ccc'})

review_controls = widgets.HBox([load_btn, prev_btn, next_btn, save_label_btn, delete_btn])
review_ui = widgets.VBox([review_controls, idx_label, review_label_dd, review_img, review_out])
display(review_ui)

file_list = []
cur_idx = -1

def review_log(msg):
    with review_out:
        print(msg)

def read_manifest():
    rows = []
    if MANIFEST.exists():
        with open(MANIFEST, 'r', newline='') as f:
            reader = csv.DictReader(f)
            for r in reader:
                rows.append(r)
    return rows

def write_manifest(rows):
    with open(MANIFEST, 'w', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=['filepath','label','timestamp'])
        writer.writeheader()
        for r in rows:
            writer.writerow(r)

def load_list(_):
    global file_list, cur_idx
    rows = read_manifest()
    file_list = rows  # list of dicts with filepath/label/timestamp
    cur_idx = 0 if file_list else -1
    review_log(f'[INFO] Loaded {len(file_list)} records from manifest.')
    show_current()

def show_current():
    global cur_idx
    if cur_idx < 0 or cur_idx >= len(file_list):
        idx_label.value = 'Index: - / -'
        review_img.value = b''
        return
    idx_label.value = f'Index: {cur_idx+1} / {len(file_list)}'
    rec = file_list[cur_idx]
    fpath = rec['filepath']
    review_label_dd.value = rec['label'] if rec['label'] in review_label_dd.options else review_label_dd.options[0]
    if os.path.exists(fpath):
        img = cv2.imread(fpath)
        # Resize preview to fit widget if needed
        if img is not None:
            preview = cv2.resize(img, (640, 360)) if img.shape[1] != 640 else img
            ret, jpg = cv2.imencode('.jpg', preview, [int(cv2.IMWRITE_JPEG_QUALITY), 90])
            if ret:
                review_img.value = jpg.tobytes()
    else:
        review_log(f'[WARN] File missing: {fpath}')
        review_img.value = b''

def prev_rec(_):
    global cur_idx
    if cur_idx > 0:
        cur_idx -= 1
        show_current()

def next_rec(_):
    global cur_idx
    if cur_idx < len(file_list)-1:
        cur_idx += 1
        show_current()

def save_label(_):
    global file_list, cur_idx
    if cur_idx < 0: return
    new_label = review_label_dd.value
    rec = file_list[cur_idx]
    old_path = Path(rec['filepath'])
    ts = rec['timestamp']
    # Move file to new label folder if changed
    new_dir = DATA_ROOT / new_label
    new_dir.mkdir(parents=True, exist_ok=True)
    new_path = new_dir / Path(old_path.name)
    if old_path.exists() and str(new_path) != str(old_path):
        try:
            old_path.replace(new_path)  # move
        except Exception as e:
            review_log(f'[ERROR] Could not move file: {e}')
            return
    # Update manifest entry
    rec['filepath'] = str(new_path)
    rec['label'] = new_label
    # Write back manifest
    write_manifest(file_list)
    review_log(f'[SAVED] Updated label to "{new_label}" for {new_path}')
    show_current()

def delete_image(_):
    global file_list, cur_idx
    if cur_idx < 0: return
    rec = file_list[cur_idx]
    fpath = Path(rec['filepath'])
    # Remove file
    if fpath.exists():
        try:
            fpath.unlink()
        except Exception as e:
            review_log(f'[ERROR] Could not delete file: {e}')
            return
    # Remove from list and rewrite manifest
    del file_list[cur_idx]
    write_manifest(file_list)
    review_log(f'[DELETED] {fpath}')
    # Adjust index and show
    if cur_idx >= len(file_list):
        cur_idx = len(file_list)-1
    show_current()

load_btn.on_click(load_list)
prev_btn.on_click(prev_rec)
next_btn.on_click(next_rec)
save_label_btn.on_click(save_label)
delete_btn.on_click(delete_image)



## 3) Export Summary (Optional)
Quick check: how many images per label?


In [ ]:

import pandas as pd

df = pd.read_csv(MANIFEST)
summary = df['label'].value_counts().rename_axis('label').reset_index(name='count')
summary



---

# 4) Bounding Box Labeler (YOLO format)

Use this section to **draw bounding boxes** on your captured images for **object detection**.  
It saves YOLO-format `.txt` files in `datasets/bbox_labels/` and a `datasets/labels.txt` file listing class names.

**Controls**
- Mouse: click–drag to draw a new box; drag corners later to adjust (basic edit supported)
- Keys: `1..9` choose class • `u` undo • `d` delete box under cursor • `n/p` next/prev image • `s` save • `q` quit



## 4.1 Configure Classes
Edit `CLASS_NAMES` so indices map to class IDs.


In [ ]:

from pathlib import Path
import cv2, json
import numpy as np

# Folder with images saved by this notebook:
IMG_ROOT = Path('datasets/raw')

# YOLO label output folder:
LBL_ROOT = Path('datasets/bbox_labels')
LBL_ROOT.mkdir(parents=True, exist_ok=True)

# Class list: index = class id (0-based)
CLASS_NAMES = ['yellow', 'purple', 'background_obj']  # <-- edit as needed
(Path('datasets') / 'labels.txt').write_text('\n'.join(CLASS_NAMES))

IMG_EXTS = ('.jpg', '.jpeg', '.png')
def sorted_images():
    imgs = []
    for p in IMG_ROOT.rglob('*'):
        if p.suffix.lower() in IMG_EXTS:
            imgs.append(p)
    return sorted(imgs)

IMAGES = sorted_images()
print('Found images:', len(IMAGES))



## 4.2 Launch the Annotator
This opens a native OpenCV window. Close with `q`.


In [ ]:

def yolo_save(lbl_path, boxes, cls_ids, W, H):
    with open(lbl_path, 'w') as f:
        for (x1,y1,x2,y2), c in zip(boxes, cls_ids):
            x1, y1 = max(0,x1), max(0,y1)
            x2, y2 = min(W-1,x2), min(H-1,y2)
            w, h = x2-x1, y2-y1
            xc, yc = x1 + w/2.0, y1 + h/2.0
            f.write(f"{c} {xc/W:.6f} {yc/H:.6f} {w/W:.6f} {h/H:.6f}\n")

def yolo_load(lbl_path, W, H):
    boxes, cls_ids = [], []
    if not lbl_path.exists(): return boxes, cls_ids
    for line in lbl_path.read_text().strip().splitlines():
        parts = line.split()
        if len(parts) != 5: continue
        c = int(parts[0]); xc=float(parts[1])*W; yc=float(parts[2])*H; w=float(parts[3])*W; h=float(parts[4])*H
        x1=int(xc-w/2); y1=int(yc-h/2); x2=int(xc+w/2); y2=int(yc+h/2)
        boxes.append([x1,y1,x2,y2]); cls_ids.append(c)
    return boxes, cls_ids

def hit_test(pt, boxes, tol=6):
    x,y = pt
    for i,(x1,y1,x2,y2) in enumerate(boxes):
        if x1-tol <= x <= x2+tol and y1-tol <= y <= y2+tol:
            return i
    return -1

def clamp_box(x1,y1,x2,y2,W,H):
    x1 = max(0, min(W-1, x1)); x2 = max(0, min(W-1, x2))
    y1 = max(0, min(H-1, y1)); y2 = max(0, min(H-1, y2))
    if x2 < x1: x1,x2 = x2,x1
    if y2 < y1: y1,y2 = y2,y1
    return [x1,y1,x2,y2]

def draw_overlay(img, boxes, cls_ids, cur_class_id, hover_idx):
    out = img.copy()
    cv2.putText(out, f"Class: {cur_class_id} ({CLASS_NAMES[cur_class_id]})", (10,25),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)
    cv2.putText(out, "1-9 class | u undo | d delete | n/p next/prev | s save | q quit", (10,55),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 2)
    for i,(x1,y1,x2,y2) in enumerate(boxes):
        color = (0,255,0) if i != hover_idx else (0,0,255)
        cv2.rectangle(out,(x1,y1),(x2,y2),color,2)
        name = CLASS_NAMES[cls_ids[i]] if 0 <= cls_ids[i] < len(CLASS_NAMES) else str(cls_ids[i])
        cv2.putText(out, name, (x1, max(0,y1-5)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
    return out

def annotate_images():
    IMGS = IMAGES
    if not IMGS:
        print('[INFO] No images found under', IMG_ROOT.resolve())
        return
    idx = 0
    cur_class = 0
    win = 'BBox Labeler'
    cv2.namedWindow(win, cv2.WINDOW_NORMAL)

    boxes, cls_ids = [], []
    drawing = False
    start_pt = None
    hover_idx = -1

    def load_current():
        nonlocal img, H, W, boxes, cls_ids
        img_path = IMGS[idx]
        img = cv2.imread(str(img_path))
        if img is None:
            print('[WARN] Could not read', img_path)
            boxes, cls_ids = [], []
            return
        H, W = img.shape[:2]
        lbl_path = LBL_ROOT / (img_path.stem + '.txt')
        boxes, cls_ids = yolo_load(lbl_path, W, H)

    def on_mouse(evt,x,y,flags,param):
        nonlocal drawing, start_pt, hover_idx, boxes, cls_ids
        if evt == cv2.EVENT_MOUSEMOVE:
            hover_idx = hit_test((x,y), boxes)
        elif evt == cv2.EVENT_LBUTTONDOWN:
            drawing = True; start_pt = (x,y)
        elif evt == cv2.EVENT_LBUTTONUP:
            if drawing and start_pt is not None:
                b = clamp_box(start_pt[0], start_pt[1], x, y, W, H)
                if abs(b[2]-b[0])>5 and abs(b[3]-b[1])>5:
                    boxes.append(b); cls_ids.append(cur_class)
            drawing = False; start_pt=None

    load_current()
    cv2.setMouseCallback(win, on_mouse)

    while True:
        disp = draw_overlay(img, boxes, cls_ids, cur_class, hover_idx)
        cv2.imshow(win, disp)
        k = cv2.waitKey(20) & 0xFF

        if k in [ord(str(x)) for x in range(10)]:  # 1..9 select classes 0..8
            num = int(chr(k))
            if 1 <= num <= min(9, len(CLASS_NAMES)):
                cur_class = num-1
        elif k == ord('u'):
            if boxes: boxes.pop(); cls_ids.pop()
        elif k == ord('d'):
            if 0 <= hover_idx < len(boxes):
                boxes.pop(hover_idx); cls_ids.pop(hover_idx); hover_idx=-1
        elif k == ord('s'):
            img_path = IMGS[idx]
            yolo_save(LBL_ROOT / (img_path.stem + '.txt'), boxes, cls_ids, W, H)
            print('[SAVED]', LBL_ROOT / (img_path.stem + '.txt'))
        elif k == ord('n'):
            idx = min(idx+1, len(IMGS)-1); load_current()
        elif k == ord('p'):
            idx = max(0, idx-1); load_current()
        elif k == ord('q') or k == 27:
            break

    cv2.destroyAllWindows()

# Run the annotator with:
# annotate_images()



## 4.3 (Optional) Export COCO JSON
Convert YOLO labels to a `datasets/coco_annotations.json` file.


In [ ]:

from PIL import Image

def yolo_to_coco():
    images = []
    annotations = []
    categories = [{"id": i, "name": n} for i, n in enumerate(CLASS_NAMES)]
    ann_id = 1
    for img_id, img_path in enumerate(IMAGES, start=1):
        if not img_path.exists(): 
            continue
        with Image.open(img_path) as im:
            W, H = im.size
        images.append({
            "id": img_id,
            "file_name": str(img_path.relative_to(IMG_ROOT.parent)),
            "width": W, "height": H
        })
        lbl_path = LBL_ROOT / (img_path.stem + '.txt')
        if not lbl_path.exists():
            continue
        for line in lbl_path.read_text().splitlines():
            parts = line.strip().split()
            if len(parts) != 5: 
                continue
            c = int(parts[0])
            xc = float(parts[1]) * W
            yc = float(parts[2]) * H
            w  = float(parts[3]) * W
            h  = float(parts[4]) * H
            x1 = xc - w/2
            y1 = yc - h/2
            annotations.append({
                "id": ann_id,
                "image_id": img_id,
                "category_id": c,
                "bbox": [x1, y1, w, h],
                "area": float(w*h),
                "iscrowd": 0
            })
            ann_id += 1
    coco = {"images": images, "annotations": annotations, "categories": categories}
    out = Path('datasets/coco_annotations.json')
    out.write_text(json.dumps(coco, indent=2))
    return out

# Generate file with:
# yolo_to_coco()



---

# 5) Preprocess Images for FTC-ML (300×300)

FTC-ML resizes all inputs to **300×300** during training.  
This step makes sure your dataset matches that expectation.

- Creates a `datasets/processed_300/` folder
- Resizes each image from `datasets/raw/` (letterboxing to keep aspect ratio)
- Copies YOLO labels if available (adjusted to new size)


In [ ]:

from pathlib import Path
import cv2, shutil

RAW_DIR = Path('datasets/raw')
PROC_DIR = Path('datasets/processed_300')
PROC_DIR.mkdir(parents=True, exist_ok=True)

TARGET = 300

def resize_and_pad(img, size=300):
    h, w = img.shape[:2]
    scale = size / max(h, w)
    new_w, new_h = int(w*scale), int(h*scale)
    resized = cv2.resize(img, (new_w, new_h))
    # pad to square
    delta_w, delta_h = size - new_w, size - new_h
    top, bottom = delta_h//2, delta_h - (delta_h//2)
    left, right = delta_w//2, delta_w - (delta_w//2)
    color = [0,0,0]
    new_img = cv2.copyMakeBorder(resized, top, bottom, left, right, cv2.BORDER_CONSTANT, value=color)
    return new_img, scale, left, top

out_manifest = PROC_DIR / 'manifest.csv'
with open(out_manifest, 'w') as mf:
    mf.write('filepath,label,timestamp\n')

count = 0
for img_path in RAW_DIR.rglob('*.jpg'):
    rel = img_path.relative_to(RAW_DIR)
    out_path = PROC_DIR / rel
    out_path.parent.mkdir(parents=True, exist_ok=True)
    img = cv2.imread(str(img_path))
    if img is None: continue
    new_img, scale, dx, dy = resize_and_pad(img, TARGET)
    cv2.imwrite(str(out_path), new_img)
    count += 1
print(f'[DONE] Processed {count} images -> {PROC_DIR}')



---
# 5) FTC‑ML 300×300 Export (letterboxed)

FTC‑ML training resizes inputs to **300×300**. Use these helpers to **export copies** of your dataset that are already letterboxed to 300×300 so your preview/testing matches what the toolchain sees.

- **Classification export**: reads `datasets/raw/manifest.csv` and writes images to `datasets/ftcml_300/<label>/...`
- **Detection export**: reads images under `datasets/raw/...` and YOLO labels in `datasets/bbox_labels/` and writes **images + adjusted YOLO labels** to `datasets/ftcml_300_bbox/`


In [ ]:

from pathlib import Path
import cv2, csv, os
import numpy as np
import pandas as pd

RAW_ROOT = Path('datasets/raw')
MANIFEST = RAW_ROOT / 'manifest.csv'

EX_CLF_ROOT = Path('datasets/ftcml_300')
EX_BBOX_IMG = Path('datasets/ftcml_300_bbox/images')
EX_BBOX_LBL = Path('datasets/ftcml_300_bbox/labels')
for p in [EX_CLF_ROOT, EX_BBOX_IMG, EX_BBOX_LBL]:
    p.mkdir(parents=True, exist_ok=True)

def letterbox(img, size=(300,300)):
    H, W = img.shape[:2]
    tw, th = size[0], size[1]
    scale = min(tw / W, th / H)
    nw, nh = int(round(W * scale)), int(round(H * scale))
    resized = cv2.resize(img, (nw, nh), interpolation=cv2.INTER_AREA)
    # Create padded canvas (black)
    canvas = np.zeros((th, tw, 3), dtype=img.dtype)
    x0 = (tw - nw) // 2
    y0 = (th - nh) // 2
    canvas[y0:y0+nh, x0:x0+nw] = resized
    return canvas, scale, x0, y0, (W, H)

def export_classification_300():
    if not MANIFEST.exists():
        print("[ERROR] No manifest at", MANIFEST)
        return
    df = pd.read_csv(MANIFEST)
    counts = {}
    for _, row in df.iterrows():
        fpath = Path(row['filepath'])
        label = str(row['label'])
        if not fpath.exists():
            continue
        img = cv2.imread(str(fpath))
        if img is None: 
            continue
        out_img, *_ = letterbox(img, (300,300))
        out_dir = EX_CLF_ROOT / label
        out_dir.mkdir(parents=True, exist_ok=True)
        out_path = out_dir / fpath.name
        cv2.imwrite(str(out_path), out_img, [int(cv2.IMWRITE_JPEG_QUALITY), 95])
        counts[label] = counts.get(label, 0) + 1
    print("[DONE] Exported classification set to", EX_CLF_ROOT.resolve())
    return counts

def export_detection_300():
    # expects YOLO labels in datasets/bbox_labels/<stem>.txt
    IMG_EXTS = ('.jpg','.jpeg','.png')
    raw_images = []
    for p in RAW_ROOT.rglob('*'):
        if p.suffix.lower() in IMG_EXTS:
            raw_images.append(p)
    raw_images = sorted(raw_images)
    converted = 0
    for img_path in raw_images:
        img = cv2.imread(str(img_path))
        if img is None: 
            continue
        boxed, scale, pad_x, pad_y, (W,H) = letterbox(img, (300,300))
        # Write image
        out_img_path = EX_BBOX_IMG / img_path.name
        cv2.imwrite(str(out_img_path), boxed, [int(cv2.IMWRITE_JPEG_QUALITY), 95])
        # Read YOLO labels (if exist), transform to new coords normalized to 300x300
        lbl_in = Path('datasets/bbox_labels') / (img_path.stem + '.txt')
        lbl_out = EX_BBOX_LBL / (img_path.stem + '.txt')
        if lbl_in.exists():
            lines_out = []
            txt = lbl_in.read_text().strip().splitlines()
            for line in txt:
                parts = line.strip().split()
                if len(parts) != 5:
                    continue
                c = int(parts[0])
                # Denormalize from original W,H
                xc = float(parts[1]) * W
                yc = float(parts[2]) * H
                w  = float(parts[3]) * W
                h  = float(parts[4]) * H
                # Apply scale + padding
                xc2 = xc * scale + pad_x
                yc2 = yc * scale + pad_y
                w2  = w  * scale
                h2  = h  * scale
                # Normalize to 300x300
                xc_n = xc2 / 300.0
                yc_n = yc2 / 300.0
                w_n  = w2  / 300.0
                h_n  = h2  / 300.0
                lines_out.append(f"{c} {xc_n:.6f} {yc_n:.6f} {w_n:.6f} {h_n:.6f}")
            lbl_out.write_text("\n".join(lines_out))
        converted += 1
    print(f"[DONE] Exported detection set to {EX_BBOX_IMG.parent.resolve()} with {converted} images.")
    return converted

# Example usage:
# export_classification_300()
# export_detection_300()



### 5.1 Quick checks
Run these to verify counts and spot-check sizes.


In [ ]:

# Export both (run as needed)
# counts = export_classification_300()
# total = export_detection_300()

# Summaries
import os
from collections import Counter

def count_files(root):
    c = Counter()
    for p in Path(root).rglob('*.jpg'):
        if p.parent.name not in ['images']:  # for classification buckets
            c[p.parent.name] += 1
    return c

print("Classification buckets:", count_files(EX_CLF_ROOT))
print("Detection images:", len(list(EX_BBOX_IMG.glob('*.jpg'))), "labels:", len(list(EX_BBOX_LBL.glob('*.txt'))))
